In [13]:
import os
from dotenv import load_dotenv

load_dotenv()

os.environ["OPENAI_MODEL_NAME"] = "gpt-4o"

In [14]:
from crewai import Crew, Agent, Task

international_chef = Agent(
    role="International Chef",
    goal="Create ethnic cuisine recipies that are easy to cook at home",
    backstory="""
    You are an famous chef that specializes in couisine from countries all around the world.
    You know how to cook the most traditional dishes from all cultures but you also know how to adapt them for people to be able to cook them at home.
    """,
    verbose=True,
    allow_delegation=False,
)

healthy_chef = Agent(
    role="Healthy Chef",
    goal="Turn any recipe into a healthy vegetarian recipe that is easy to cook with home ingredients.",
    backstory="""
    You are a chef specialized in healthy cooking.
    You can take any recipe and change the ingredients to make it vegetarian friendly without loosing the escense of the dish and what makes it delicious.
    """,
    verbose=True,
    allow_delegation=False,
)

In [15]:
normal_recipe = Task(
    description="Come up with a {dish} that serves {people} people.",
    agent=international_chef,
    expected_output="Your answer MUST have three sections, the ingredients required with their quantities, the preparation instructions and serving suggestions",
    output_file="normal_recipe.md",
)

healthy_recipe = Task(
    description="Replace the ingredients of a recipe to make it vegetarian without making it less delicious, adjust if needed.",
    agent=healthy_chef,
    expected_output="Your answer MUST have four sections, the ingredients required with their quantities, the preparation instructions, serving suggestions and an explanation of the replaced ingredients.",
    output_file="healthy_recipe.md",
)

In [16]:
crew = Crew(
    tasks=[
        normal_recipe,
        healthy_recipe,
    ],
    agents=[
        international_chef,
        healthy_chef,
    ],
    verbose=2,
)

result = crew.kickoff(
    inputs=dict(
        dish="Greek dinner",
        people="5",
    )
)

2024-05-30 15:36:12,016 - 8718023360 - __init__.py-__init__:518 - WARNING: Overriding of current TracerProvider is not allowed


 [DEBUG]: == Working Agent: International Chef
 [INFO]: == Starting Task: Come up with a Greek dinner that serves 5 people.


> Entering new CrewAgentExecutor chain...
I now can give a great answer.
Final Answer: 

### Greek Dinner for 5 People

---

#### Ingredients Required:

**Appetizer: Greek Salad**
- 4 large tomatoes, chopped
- 1 cucumber, sliced
- 1 red onion, thinly sliced
- 1 green bell pepper, sliced
- 200g feta cheese, cubed
- 100g kalamata olives
- 2 tbsp extra-virgin olive oil
- 1 tbsp red wine vinegar
- 1 tsp dried oregano
- Salt and black pepper to taste

**Main Course: Moussaka**
- 2 large eggplants, sliced
- 500g ground beef or lamb
- 1 large onion, chopped
- 2 cloves garlic, minced
- 1 can (400g) crushed tomatoes
- 2 tbsp tomato paste
- 1 tsp cinnamon
- 1 tsp allspice
- 1/2 cup red wine (optional)
- 2 tbsp olive oil
- Salt and pepper to taste
- 3 tbsp all-purpose flour
- 2 cups milk
- 1 cup grated Parmesan cheese
- 2 eggs, beaten
- 1/4 tsp ground nutmeg

**Dessert: Ba